<a href="https://colab.research.google.com/github/Inshalkhan/Data-Science-and-Decision-Making/blob/master/Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [0]:
df = pd.read_csv('jester-data-1.csv',header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [0]:
df=df.drop(df.columns[0],1)
print(df)

         1      2      3      4     5    ...    96     97     98     99     100
0      -7.82   8.79  -9.66  -8.16 -7.52  ...  99.00  -5.63  99.00  99.00  99.00
1       4.08  -0.29   6.36   4.37 -2.38  ...  -2.14   3.06   0.34  -4.32   1.07
2      99.00  99.00  99.00  99.00  9.03  ...  99.00  99.00  99.00  99.00  99.00
3      99.00   8.35  99.00  99.00  1.80  ...  99.00  99.00  99.00  99.00  99.00
4       8.50   4.61  -4.17  -5.39  1.36  ...   1.55   3.11   6.55   1.80   1.60
...      ...    ...    ...    ...   ...  ...    ...    ...    ...    ...    ...
24978   0.44   7.43   9.08   2.33  3.20  ...   9.03   6.55   8.69   8.79   7.43
24979   9.13  -8.16   8.59   9.08  0.87  ...  -8.20  -7.23  -8.59   9.13   8.45
24980  99.00  99.00  99.00  99.00 -7.77  ...  99.00  99.00  99.00  99.00  99.00
24981  99.00  99.00  99.00  99.00 -9.71  ...  99.00  99.00  99.00  99.00  99.00
24982   2.43   2.67  -3.98   4.27 -2.28  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 100 columns]


In [0]:
arr = df.values
print(arr.shape)

(24983, 100)


In [0]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [0]:
arr[idx[0][0], idx[1][0]]

-9.37

In [0]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
n_latent_factors = 15

user_ratings = df.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_arr[user_id][item_id]
                if (rating!=99):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 10) == 0:
        print(mse)

In [0]:
sgd()

25.87472414006913
24.944952765801517
24.894599011813728
24.848567444641976
24.804642427386092
24.760906196522406
24.715385617391696
24.665725189232766
24.608793381050052
24.540179384181283
24.453541977401944
24.339811056653453
24.186350312664004
23.976431218313262
23.689803907790534
23.305697372418493
22.80969306679999
22.20431753572768
21.518848698722675
20.809262482141502
20.142154035473414
19.569944365424142
19.114899862170798
18.771280864223513
18.5184796166776
18.33325958155602
18.195978113079548
18.092018755014983
18.01109815457415
17.946113557065534
17.892153550560888
17.845785113633124
17.80457057320119
17.766746277890192
17.731009034713416
17.69637344720259
17.662076054761595
17.62751064973585
17.592184604096534
17.55568953666943
17.517681904929283
17.477870560156425
17.43600924208756
17.39189258789125
17.345354606717045
17.29626880314632
17.24454927477844
17.190152200116206
17.13307720205121
17.073368141033146


In [0]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[-3.902, -4.015, -5.277, ..., -3.184, -5.286, -3.534],
       [ 4.819,  4.381,  4.945, ...,  2.617,  1.762,  3.724],
       [ 6.278,  6.165,  6.403, ...,  5.648,  5.517,  6.122],
       ...,
       [-0.892, -1.422, -1.448, ..., -1.015, -1.562, -1.488],
       [-0.726, -1.586, -1.935, ..., -2.83 , -3.52 , -2.526],
       [ 1.212,  3.214,  1.397, ...,  4.691,  1.853,  4.709]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [0]:
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -3.9023244370857215)","(8.79, -4.0146105673443575)","(-9.66, -5.277150182740173)","(-8.16, -7.869709214284014)","(-7.52, -2.2482151199683256)","(-8.5, -2.8286435303885296)","(-9.85, -3.2692361102365903)","(4.17, -2.8335031386109626)","(-8.98, -7.545073932481512)","(-4.76, -3.978935314822791)","(-8.5, -2.9292221644036913)","(-6.75, -2.871563143799581)","(-7.18, -4.70422165077178)","(8.45, -2.7533205286220244)","(-7.18, -4.399640337773629)","(-7.52, -6.049431301319912)","(-7.43, -3.151282818309432)","(-9.81, -3.8644560060079414)","(-9.85, -2.5459542684488747)","(-9.85, -4.380206415272306)","(-9.37, -1.182149157501533)","(1.5, -4.300799270603249)","(-4.37, -5.616146049445155)","(-9.81, -8.50466091249775)","(-8.5, -4.438502188580649)","(1.12, -2.771461590809576)","(7.82, 0.8145634881393032)","(2.86, -2.669775555642486)","(9.13, 1.0651020785417833)","(-7.43, -6.861117634191254)","(2.14, -0.945758809357081)","(-4.08, 1.0222214552221336)","(-9.08, -7.822441030430669)","(7.82, -3.482313465458237)","(5.05, 0.4309885104383756)","(4.95, 1.1553800679440223)","(-9.17, -7.383648194951515)","(-8.4, -3.598743608807099)","(-8.4, -3.2884236536526994)","(-8.4, -3.7896313357281586)",...,"(8.59, -1.0416505429198257)","(3.59, -0.3049065008240401)","(-6.84, -4.948184645496371)","(-9.03, -6.65422634237765)","(2.82, -0.9903274373112452)","(-1.36, -0.7334991990362448)","(-9.08, -7.334029221293042)","(8.3, -0.3308292086045187)","(5.68, -0.36814568806650666)","(-4.81, -4.251781470863876)","(99.0, -6.371127809485021)","(99.0, -1.0034098911557416)","(99.0, -3.824300015577709)","(99.0, -7.225727475317901)","(99.0, -5.079607875983298)","(99.0, -0.7557054308225434)","(99.0, -4.058438159337591)","(-9.42, -2.8414681605459093)","(99.0, -4.791306862219849)","(99.0, -3.6470776746406126)","(99.0, -2.7593891559606245)","(-7.72, -4.552080437985196)","(99.0, -1.2907109469281142)","(99.0, -5.079195484998247)","(99.0, -4.681040331677805)","(99.0, -5.209392978025537)","(99.0, -1.9583341189607875)","(99.0, -2.265561139592735)","(99.0, 0.7190108849588026)","(99.0, -4.69784816057077)","(2.82, -2.520566137888561)","(99.0, -3.284723273717294)","(99.0, -1.1177490376908625)","(99.0, -4.506722227807987)","(99.0, -3.601719683662133)","(99.0, -3.4514608603945773)","(-5.63, -2.7539826546982096)","(99.0, -3.1842538096281685)","(99.0, -5.286138254581454)","(99.0, -3.5342677516573526)"
1,"(4.08, 4.819038707397277)","(-0.29, 4.380607834284293)","(6.36, 4.9445378431816875)","(4.37, 3.9951933318877773)","(-2.38, 1.6750294918459454)","(-9.66, 3.0644570225847785)","(-0.73, 0.9398552175566581)","(-5.34, 0.4893674140467371)","(8.88, 4.328241883822326)","(9.22, 5.018750805785097)","(6.75, 4.321160822167846)","(8.64, 3.793983448198218)","(4.42, 0.6279076164628768)","(7.43, 3.8139913454328944)","(4.56, 0.02655747911787925)","(-0.97, -0.6288609634687832)","(4.66, 0.19634577251929397)","(-0.68, 0.9948908355350813)","(3.3, 0.15080455607450888)","(-1.21, -0.2023663724785705)","(0.87, 3.80821598390658)","(8.64, 5.054927465131034)","(8.35, 3.825178072459814)","(9.17, 4.2393827586591515)","(0.05, 3.930829355505655)","(7.57, 3.507034867534522)","(4.71, 3.119724440027277)","(0.87, 2.2132736250121634)","(-0.39, 2.924757548691981)","(6.99, 3.3623450491983653)","(6.5, 3.5591822734288865)","(-0.92, 1.9566388454874566)","(7.14, 3.973465714180414)","(9.03, 4.156479010131228)","(-1.8, 2.888904999151549)","(0.73, 2.804872423594154)","(7.09, 3.3448851611407573)","(3.4, 4.015150269612275)","(-0.87, 3.337641347953141)","(7.91, 3.9561930375535526)",...,"(-6.7, 2.9095187644215152)","(-3.35, 2.4734661761100556)","(-9.03, 2.2951935830654415)","(4.47, 4.127901612138213)","(4.08, 3.8855627564933974)","(-3.83, 2.9259128629048208)","(8.74, 4.483228844753945)","(1.12, 1.936235086